In [1]:
using Laplacians

objc[23117]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[23117]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[23117]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[23117]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


In [15]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgVBLASMatNorm (generic function with 1 method)

In [16]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [17]:
a = readFromFile("../graphs/se_1/graph.mtx");

In [18]:
tree = readFromFile("../graphs/se_1/tree.mtx");

In [19]:
x = readFromFile("../graphs/se_1/x.vec");

In [20]:
# b = readFromFile("../graphs/se_1/b.vec");
b = lap(a) * x;

In [21]:
norm(lap(a) * x - b) / norm(b)

0.0

In [22]:
la = lap(a);

In [23]:
ts = treeSolver(tree)

F = function(b)
    return ts(b - mean(b))
end

(anonymous function)

In [42]:
@time myx64 = pcg(la, b, F, tol=1e-6, maxits=100000, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)

PCG stopped after: 1232 iterations with relative error 8.16951834111911e-7.
  0.809712 seconds (16.66 M allocations: 701.473 MB, 10.39% gc time)


8.169518351480716e-7

In [ ]:
dbg = []

for precision in [64,128,512,1024]
    hp = precision
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    tsHP = treeSolver(treeHP)

    FHP = function(b)
        return tsHP(b - mean(b))
    end
    
    push!(dbg, "======= $(hp) bit precision")
    @time myxHP = pcg(laHP, bHP, FHP, tol=1e-6, maxits=100000, verbose=true);
    norm(laHP * myxHP - bHP) / norm(bHP)
    push!(dbg, "")
    push!(dbg, "")
    
end

In [ ]:
writeToFile("../graphs/se_1/_log_julia.txt", dbg)